## Day 8: Handheld Halting 

Your flight to the major airline hub reaches cruising altitude without incident. While you consider checking the in-flight menu 
for one of those drinks that come with a little umbrella, you are interrupted by the kid sitting next to you.

Their handheld game console won't turn on! They ask if you can take a look.

You narrow the problem down to a strange infinite loop in the boot code (your puzzle input) of the device. 
You should be able to fix it, but first you need to be able to run the code in isolation.

The boot code is represented as a text file with one instruction per line of text. Each instruction consists of an operation 
(acc, jmp, or nop) and an argument (a signed number like +4 or -20).

acc increases or decreases a single global value called the accumulator by the value given in the argument. For example, acc +7 would increase the accumulator 
by 7. The accumulator starts at 0. After an acc instruction, the instruction immediately below it is executed next.
jmp jumps to a new instruction relative to itself. The next instruction to execute is found using the argument as an offset from the jmp instruction; 
for example, jmp +2 would skip the next instruction, jmp +1 would continue to the instruction immediately below it, and jmp -20 
would cause the instruction 20 lines above to be executed next.
nop stands for No OPeration - it does nothing. The instruction immediately below it is executed next.
For example, consider the following program:
```
nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
```
These instructions are visited in this order:
```
nop +0  | 1
acc +1  | 2, 8(!)
jmp +4  | 3
acc +3  | 6
jmp -3  | 7
acc -99 |
acc +1  | 4
jmp -4  | 5
acc +6  |
```
First, the nop +0 does nothing. Then, the accumulator is increased from 0 to 1 (acc +1) 
and jmp +4 sets the next instruction to the other acc +1 near the bottom. 
After it increases the accumulator from 1 to 2, jmp -4 executes, setting the next instruction to the only acc +3. It sets the accumulator to 5, 
and jmp -3 causes the program to continue back at the first acc +1.

This is an infinite loop: with this sequence of jumps, the program will run forever. The moment the program tries to run any instruction a second time, 
you know it will never terminate.

Immediately before the program would run an instruction a second time, the value in the accumulator is 5.

Run your copy of the boot code. Immediately before any instruction is executed a second time, what value is in the accumulator?

In [ ]:
import re

In [ ]:
instructions = list(map(str, open('../data/day8_input.txt', 'r').read().split('\n')))

In [ ]:
instructions[0:5]

['acc +7', 'acc +23', 'acc +41', 'jmp +173', 'acc -17']

In [ ]:
def run_program(instructions):
    d_count = {} # empty dictionary to store counts of list index. This will break loop if it sees the index is already in here while on the index.
    acc = 0
    index = 0
    while index not in d_count:
        if index == len(instructions) - 1:
            finished = 'y'
            return acc, finished
        else:
            d_count[index] = 1
            digit = re.findall('([+\-]\d+)', instructions[index])[0]
            move = instructions[index][0:3]
            if move == 'acc':
                acc+= int(digit)
                index +=1
            elif move == 'jmp':
                index += int(digit)
            else:
                index +=1
    finished = 'n'
    return acc, finished


In [ ]:
# Answer to part 1 (the string after implies that it would have stayed in an infinite loop (didn't finish))
run_program(instructions)

(1563, 'n')

## Part 2

After some careful analysis, you believe that exactly one instruction is corrupted.

Somewhere in the program, either a jmp is supposed to be a nop, or a nop is supposed to be a jmp. (No acc instructions were harmed in the corruption of this boot code.)

The program is supposed to terminate by attempting to execute an instruction immediately after the last instruction in the file. By changing exactly one jmp or nop, you can repair the boot code and make it terminate correctly.

For example, consider the same program from above:
```
nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
```
If you change the first instruction from nop +0 to jmp +0, it would create a single-instruction infinite loop, never leaving that instruction. 
If you change almost any of the jmp instructions, the program will still eventually find another jmp instruction and loop forever.

However, if you change the second-to-last instruction (from jmp -4 to nop -4), the program terminates! The instructions are visited in this order:
```
nop +0  | 1
acc +1  | 2
jmp +4  | 3
acc +3  |
jmp -3  |
acc -99 |
acc +1  | 4
nop -4  | 5
acc +6  | 6
```
After the last instruction (acc +6), the program terminates by attempting to run the instruction below the last instruction in the file. 
With this change, after the program terminates, the accumulator contains the value 8 (acc +1, acc +1, acc +6).

Fix the program so that it terminates normally by changing exactly one jmp (to nop) or nop (to jmp). What is the value of the accumulator 
after the program terminates?

In [ ]:
"""An issue here is that I'll need tons of sets of instructions here, the ones for the cases where it's a switch from 'jmp' to 'nop' and the others in cases
where it's a switch from 'nop' to 'jmp'. I think at the beginning of each loop, I'll create a copy of the instructions. This seems super "expensive".
I want to talk to someone about this.

The main idea here will be to send the copy of the instructions, and the one swapped change to the other function we created, and see if it got to the end;
I'll add this new functionality to it. 
"""

def run_program_to_completion(instructions):
    acc = 0
    index = 0
    # Step 1. Change one 'jmp' to 'nop'
    jmp_indices = [i for i, x in enumerate(instructions) if "jmp" in x] # list comprehensions are the bomb!
    for index in jmp_indices:
        instructions_copy = instructions.copy()
        instructions_copy[index] = instructions_copy[index].replace("jmp","nop") # replace the 'jmp' with 'nop'
        acc, finished = run_program(instructions_copy)
        if finished == 'n':
            continue
        else:
            return acc
    
    # Step 2. Change one 'nop' to 'jmp'
    nop_indices = [i for i, x in enumerate(instructions) if "nop" in x]
    for index in nop_indices:
        instructions_copy = instructions.copy()
        instructions_copy[index] = instructions_copy[index].replace("nop","jmp") # replace the 'jmp' with 'nop'
        acc, finished = run_program(instructions_copy)
        if finished == 'n':
            continue
        else:
            return acc

In [ ]:
# Answer to part 2
run_program_to_completion(instructions)

767

## Learning from the [subreddit](http:reddit.com/r/adventofcode)

When I saw part 2, I was particularly interested in the non-brute force approach, but knew it would probably take me a lot longer to figure out so I decided
to avoid it.

I stumbled across this [post](https://www.reddit.com/r/adventofcode/comments/k8zdx3/day_8_part_2_without_bruteforce/) after I finished
with someone asking the same question. The top reply by `u/lasagnaman` is pretty fascinating.
> Let $C$ be the initial set of commands. We can compute the set WINNING of positions that will eventually lead to a victory (in $C$) in $O(N)$ time.

> Now suppose we are looking at some modified command set $C'$, where we have changed some position $i$ from 'jmp' to 'nop' (or vice versa). 
Let $j$ be the position after executing the (modified) line $i$. I claim that:

>    (Lemma 1) $j$ leads to victory in $C'$ if and only if $j$ leads to victory in $C$. This means we only need to check in $j$ is in WINNING.

> So, we can just follow along $C$; every time we see a 'nop' or 'jmp', change it to the other instruction and see if that lands you in WINNING. If so, 
finish out the rest of the alg and return acc. If not, proceed without modifying the command. This part of the algorithm also takes O(N).
Finally, let me prove Lemma 1.

> Forwards direction: Suppose $j$ leads to victory in $C'$. If $j$ eventually leads back to $i$ (in $C'$), we are in an infinite loop. Therefore, the path 
from $j$ to victory in $C'$ does not include $i$. Hence, this path is unchanged between $C$ and $C'$, so $j$ also leads to victory in $C$.
Backwards direction: Suppose $j$ leads to victory in $C$. If $j$ leads to $i$ (in $C$), then $i$ also leads to victory in $C$. But this is 
impossible since $i$ is part of the initial infinite loop in $C$. Therefore, $j$'s path to victory does not include $i$. Proceeding as in the previous 
case, we conclude that $j$ also leads to victory in $C'$. (Note that we weren't able to use the exact same argument as the previous case, because 
$i$ doesn't necessarily lead to $j$ in $C$!)



This is awesome! This means that we just have to run the loop one time, switch out 'jmp' for 'nop' (or vice-versa) and we should get the same answer!
Let's talk about this with Mauricio later today if he sees this and try it out.

